In [ ]:
%pip install yfinance
%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install statsmodels
%pip install datetime
%pip install scikit-learn
%pip install PyPortfolioOpt

In [ ]:
%pip install pandas-ta

## Load SP500 Data

In [46]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas_ta
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import warnings

In [13]:
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')
symbols_list = sp500['Symbol'].unique().tolist()

In [14]:
end_date = '2025-01-01'

start_date = pd.to_datetime(end_date) - pd.DateOffset(365*8)

In [21]:
df = yf.download(tickers=symbols_list, start=start_date, end=end_date, auto_adjust=False)
df.stack()

[*********************100%***********************]  503 of 503 completed
<ipython-input-21-5be7c2eae0f1>:2: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df.stack()


Price               Adj Close       Close        High         Low        Open  \
Date       Ticker                                                               
2017-01-03 A        43.651131   46.490002   46.750000   45.740002   45.930000   
           AAPL     26.827246   29.037500   29.082500   28.690001   28.950001   
           ABBV     43.450218   62.410000   63.029999   61.939999   62.919998   
           ABT      33.456699   39.049999   39.080002   38.340000   38.630001   
           ACGL     27.224224   28.629999   29.143333   28.506666   28.943333   
...                       ...         ...         ...         ...         ...   
2024-12-31 XYL     115.293098  116.019997  117.110001  115.570000  116.779999   
           YUM     132.877213  134.160004  134.789993  133.250000  134.089996   
           ZBH     105.404167  105.629997  106.500000  104.959999  105.910004   
           ZBRA    386.220001  386.220001  387.410004  381.750000  383.420013   
           ZTS     161.892609  162.929993  163.750000  161.869995  162.899994   

Price                   Volume  
Date       Ticker               
2017-01-03 A         1739600.0  
           AAPL    115127600.0  
           ABBV      9328200.0  
           ABT       9677300.0  
           ACGL       942900.0  
...                        ...  
2024-12-31 XYL        641600.0  
           YUM       1217100.0  
           ZBH        683300.0  
           ZBRA       327900.0  
           ZTS       1327400.0  

[989160 rows x 6 columns]

In [28]:
df = df.stack()

<ipython-input-28-94aab09ac48f>:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = df.stack()


## Calculate features and technical indicators for each stock

In [51]:
from typing_extensions import dataclass_transform
# Garman-Klass Volatility
df['garman_klass_vol']=((np.log(df['High']) - np.log(df['Low']))**2)/2 - (2*np.log(2)-1)*((np.log(df['Adj Close']) - np.log(df['Open']))**2)

# RSI
df['rsi'] = df.groupby(level=1)['Adj Close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

# BBL
df['bb_low'] = df.groupby(level=1)['Adj Close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
df['bb_mid'] = df.groupby(level=1)['Adj Close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
df['bb_high'] = df.groupby(level=1)['Adj Close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])